In [1]:
import numpy as np
import pandas as pd
import string
import re

In [2]:
with open('businessCard.txt', mode='r',encoding='utf8', errors='ignore') as f:
    text = f.read()

In [3]:
# print(text)

In [4]:
# each column is split by \t and each line is split by \n
data = list(map(lambda x:x.split('\t'), text.split('\n')))
# map(function to apply, the thing thats function is applied to)

In [5]:
df = pd.DataFrame(data[1:],columns=data[0])

In [6]:
df.head(10)

,id,text,tag
0,000.jpeg,,O
1,000.jpeg,.,O
2,000.jpeg,040-4852,B-PHONE
3,000.jpeg,"""8881,""",I-PHONE
4,000.jpeg,90309,B-PHONE
5,000.jpeg,52549,I-PHONE
6,000.jpeg,Fi,O
7,000.jpeg,/laurelsoverseaseducation,O
8,000.jpeg,@:,O
9,000.jpeg,LAURELS,B-ORG


### Cleaning Text
* Remove white space
* Remove Unwanted special characters

In [7]:
whitespace = string.whitespace
punctuation = '!"#$%&\'()*+:;<=>?[\\]^`{|}~'
tableWhitespace = str.maketrans('','',whitespace) # replace whitespace with ''
tablePunctuation = str.maketrans('','',punctuation)
# maketrans(x,y,z) args: if found x, replace with y, if found z, remove z
def cleanText(txt):
    text = str(txt)
    text = text.lower()
    removewhitespace = text.translate(tableWhitespace)
    removepunctuation = removewhitespace.translate(tablePunctuation)

    return str(removepunctuation)
# punctuation = punctuation to remove

In [8]:
# keep some characters in punctuation like @ and /
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [9]:
df['text'] = df['text'].apply(cleanText)

In [10]:
dataClean = df.query("text != '' ")
dataClean.dropna(inplace=True)

C:\Users\David\AppData\Local\Temp\ipykernel_15644\3379659824.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataClean.dropna(inplace=True)


In [11]:
dataClean.head(10)

,id,text,tag
1,000.jpeg,.,O
2,000.jpeg,040-4852,B-PHONE
3,000.jpeg,"8881,",I-PHONE
4,000.jpeg,90309,B-PHONE
5,000.jpeg,52549,I-PHONE
6,000.jpeg,fi,O
7,000.jpeg,/laurelsoverseaseducation,O
8,000.jpeg,@,O
9,000.jpeg,laurels,B-ORG
10,000.jpeg,overseas,I-ORG


In [12]:
group = dataClean.groupby(by='id')

In [13]:
cards = group.groups.keys()
cards

dict_keys(['000.jpeg', '001.jpeg', '002.jpeg', '003.jpeg', '004.jpeg', '007.jpeg', '008.jpeg', '009.jpeg', '010.jpeg', '011.jpeg', '012.jpeg', '013.jpeg', '014.jpeg', '015.jpeg', '016.jpeg', '017.jpeg', '018.jpeg', '020.jpeg', '021.jpeg', '022.jpeg', '023.jpeg', '024.jpeg', '025.jpeg', '027.jpeg', '028.jpeg', '030.jpeg', '031.jpeg', '032.jpeg', '033.jpeg', '034.jpeg', '035.jpeg', '036.jpeg', '037.jpeg', '038.jpeg', '039.jpeg', '040.jpeg', '041.jpeg', '042.jpeg', '043.jpeg', '044.jpeg', '045.jpeg', '047.jpeg', '048.jpeg', '049.jpeg', '050.jpeg', '051.jpeg', '052.jpeg', '053.jpeg', '054.jpeg', '055.jpeg', '056.jpeg', '057.jpeg', '058.jpeg', '059.jpeg', '060.jpeg', '061.jpeg', '062.jpeg', '063.jpeg', '064.jpeg', '065.jpeg', '066.jpeg', '067.jpeg', '068.jpeg', '069.jpeg', '070.jpeg', '071.jpeg', '072.jpeg', '073.jpeg', '074.jpeg', '075.jpeg', '076.jpeg', '078.jpeg', '079.jpeg', '080.jpeg', '081.jpeg', '082.jpeg', '083.jpeg', '084.jpeg', '085.jpeg', '086.jpeg', '087.jpeg', '088.jpeg', '089.

In [14]:
allCardsData = []
for card in cards:
    cardData = []
    grouparray = group.get_group(card)[['text','tag']].values
    content = '' # all text 
    annotations = {'entities':[]} # ditc: information about labels
    start = 0 # positions
    end = 0
    for text, label in grouparray:
        text = str(text)
        stringLength = len(text) + 1
    
        start = end
        end = start + stringLength
    
        if label != 'O':
            annot = (start, end-1, label)
            annotations['entities'].append(annot)
    
        content = content + text + ' '

    cardData = (content, annotations)
    allCardsData.append(cardData)

In [15]:
content

'a5 p srmrrisurres richard pretorius director '

In [16]:
annotations

{'entities': [(5, 17, 'B-ORG'),
  (18, 25, 'B-NAME'),
  (26, 35, 'I-NAME'),
  (36, 44, 'B-DES')]}

In [17]:
content.find('040-4852') + len('040-4852') # entity test

7

### Split the data into training and testing set

In [18]:
import random

In [19]:
random.shuffle(allCardsData)

In [20]:
len(allCardsData)

267

In [21]:
TrainData = allCardsData[:240]
TestData = allCardsData[240:]

In [22]:
import pickle

In [23]:
pickle.dump(TrainData, open('./data/TrainData.pickle',mode='wb'))
pickle.dump(TrainData, open('./data/TestData.pickle',mode='wb'))